In [1]:
pip install umap-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 21.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for umap-learn: filename=umap_learn-0.5.3-py3-none-any.whl size=82830 sha256=634268d85347282153c4fdc672ac7ab0a96e3ffa513ae7fed80331c8df1b2f8e
  Stored in directory: /root/.cache/pip/wheels/f4/3e/1c/596d0a463d17475af648688443fa4846fef624d1390339e7e9
  Created wheel for pynndescent: filename=pynndescent-0.5.10-py3-none-any.whl size=55640 sha256=61b3260cc3b31dfd91cd74a2942c2f6c453bf484a9a391e121fe84842084e737
  Stored in directory: /root/.cache/pip/wheels/12/f9/4d/ec5ad1c823c710fcc4473669fdcffc8891f4bc398c841af22e
Successfully built umap-learn pynndescent


Importing Necessary Files

In [2]:
import re, nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('omw-1.4')
nltk.download('wordnet')
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
import plotly.graph_objs as go
import plotly.figure_factory as ff
import umap

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package wordnet to /root/nltk_data...


Reading dataset as dataframe

In [3]:
df = pd.read_csv("Reviews.csv")
pd.set_option('display.max_colwidth', None) # Setting this so we can see the full content of cells
pd.set_option('display.max_columns', None) # to make sure we can see all the columns in output window

Converting unstructured 'Review' column to a TF-IDF matrix

In [4]:

def cleaner(review): # Cleaning reviews
    soup = BeautifulSoup(review, 'lxml') # removing HTML entities such as ‘&amp’,’&quot’,'&gt'; lxml is the html parser and shoulp be installed using 'pip install lxml'
    souped = soup.get_text()
    re1 = re.sub("[^A-Za-z]+"," ", souped) # substituting any non-alphabetic character that repeats one or more times with whitespace

    tokens = nltk.word_tokenize(re1)
    lower_case = [t.lower() for t in tokens]

    stop_words = set(stopwords.words('english'))
    filtered_result = list(filter(lambda l: l not in stop_words, lower_case))

    wordnet_lemmatizer = WordNetLemmatizer()
    lemmas = [wordnet_lemmatizer.lemmatize(t) for t in filtered_result]
    return lemmas

In [5]:
df['cleaned_review'] = df.Review.apply(cleaner)
df = df[df['cleaned_review'].map(len) > 0] # removing rows with cleaned reviews of length 0
print("Printing top 5 rows of dataframe showing original and cleaned reviews....")
print(df[['Review','cleaned_review']].head())
df['cleaned_review'] = [" ".join(row) for row in df['cleaned_review'].values] # joining tokens to create strings. TfidfVectorizer does not accept tokens as input
data = df['cleaned_review']
Y = df['Rating'] # label column
tfidf = TfidfVectorizer(min_df=0.00096 , ngram_range=(1,4)) # min_df=.00096 means that each ngram (unigram, bigram, & trigram) must be present in at least 20 documents for it to be considered as a token (23305*.00086=20). This is a clever way of feature engineering
tfidf.fit(data) # learn vocabulary of entire data
data_tfidf = tfidf.transform(data) # creating tfidf values
print("The created tokens: \n", tfidf.get_feature_names_out())
print("Shape of tfidf matrix: ", data_tfidf.shape)
print(type(data_tfidf))

Printing top 5 rows of dataframe showing original and cleaned reviews....
                                                                                                                                                                                                      Review  \
0     The Gourmet Kitchen serves the most delicious French cuisine in town! The ambience is perfect for a romantic dinner or a night out with friends. I highly recommend the escargot and the crème brûlée.   
1                              Mama's Italian Kitchen is my new favorite Italian restaurant! The pasta is always cooked perfectly, and the sauce is rich and flavorful. The garlic bread is also a must-try.   
2          Sushi Palace is the best sushi restaurant in the city. The sushi is always fresh and expertly prepared, and the miso soup is the perfect starter. The atmosphere is also very cozy and welcoming.   
3  The Green Table is the perfect spot for vegetarians and vegans. The food is always fresh an

Implementing UMAP to visualize dataset

In [6]:
u = umap.UMAP(n_components = 2, n_neighbors=50, min_dist=0.4)
x_umap = u.fit_transform(data_tfidf)

ratings = list(df['Rating'])
reviews = list(df['Review'])
data = [go.Scatter(x=x_umap[:,0], y=x_umap[:,1], mode='markers',
                    marker = dict(color=Y, colorscale='Rainbow', opacity=0.5),
                                text=[f'Rating: {a}<br>Review: {b}' for a,b in list(zip(ratings,reviews))],
                                hoverinfo='text')]

layout = go.Layout(title = 'UMAP Dimensionality Reduction', width = 1400, height = 1400,
                    xaxis = dict(title='First Dimension'),
                    yaxis = dict(title='Second Dimension'))
fig = go.Figure(data=data, layout=layout)
fig.show()

The provided code is written in Python and aims to convert the "Reviews.csv" file into a TF-IDF matrix and then visualize it using the UMAP (Uniform Manifold Approximation and Projection) dimensionality reduction technique.

First, the necessary libraries such as re, nltk, numpy, pandas, BeautifulSoup, and sklearn are imported. The code uses the BeautifulSoup library to remove HTML entities from the reviews. Then, the reviews are cleaned using the cleaner() function, which converts all text to lowercase, removes stop words, and applies lemmatization to the remaining words. The cleaned reviews are then joined and stored in a new column of the DataFrame called "cleaned_review."

Next, the TfidfVectorizer is used to create a TF-IDF matrix from the cleaned reviews. TfidfVectorizer converts a collection of raw documents to a matrix of TF-IDF features. In this case, the min_df parameter is set to 0.00096, which means that each ngram (unigram, bigram, & trigram) must be present in at least 23 documents for it to be considered as a token (2400 * 0.00096=23). This is a clever way of feature engineering.

After creating the TF-IDF matrix, the UMAP dimensionality reduction technique is applied to it. The UMAP algorithm maps high-dimensional data to a low-dimensional space while preserving the structure of the original data. In this case, UMAP is used to reduce the dimensionality of the TF-IDF matrix to 2 dimensions for visualization purposes.

Finally, the code uses Plotly to create a scatter plot of the reduced TF-IDF matrix, where each point represents a review. The color of the point represents the rating of the review, and the text associated with each point contains the actual review text and its corresponding rating.

The visualization shows that positive reviews with ratings of 4 and 5 (represented by red and yellow dots) are clustered together, while negative reviews with ratings of 1 and 2 (represented by blue and purple dots) are also clustered together. Neutral reviews with a rating of 3 (represented by green dots) are scattered throughout the plot.

----------------------------------------------------------------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------------------------------------------------------------


2. How does tuning TF-IDF hyperparameters ‘ngram_range’ and ‘min_df’ affect the TF-IDF matrix and the subsequent visualization?
The hyperparameters 'ngram_range' and 'min_df' of the TfidfVectorizer can have a significant impact on the resulting TF-IDF matrix and the visualization.


In [31]:
df['cleaned_review'] = df.Review.apply(cleaner)
df = df[df['cleaned_review'].map(len) > 0] # removing rows with cleaned reviews of length 0
print("Printing top 5 rows of dataframe showing original and cleaned reviews....")
print(df[['Review','cleaned_review']].head())
df['cleaned_review'] = [" ".join(row) for row in df['cleaned_review'].values] # joining tokens to create strings. TfidfVectorizer does not accept tokens as input
data = df['cleaned_review']
Y = df['Rating'] 
tfidf = TfidfVectorizer(min_df=0.00085, ngram_range=(1,4)) 
tfidf.fit(data) # learn vocabulary of entire data
data_tfidf = tfidf.transform(data) # creating tfidf values
print("The created tokens: \n", tfidf.get_feature_names_out())
print("Shape of tfidf matrix: ", data_tfidf.shape)
print(type(data_tfidf))

Printing top 5 rows of dataframe showing original and cleaned reviews....
                                                                                                                                                                                                      Review  \
0     The Gourmet Kitchen serves the most delicious French cuisine in town! The ambience is perfect for a romantic dinner or a night out with friends. I highly recommend the escargot and the crème brûlée.   
1                              Mama's Italian Kitchen is my new favorite Italian restaurant! The pasta is always cooked perfectly, and the sauce is rich and flavorful. The garlic bread is also a must-try.   
2          Sushi Palace is the best sushi restaurant in the city. The sushi is always fresh and expertly prepared, and the miso soup is the perfect starter. The atmosphere is also very cozy and welcoming.   
3  The Green Table is the perfect spot for vegetarians and vegans. The food is always fresh an

In [32]:
u = umap.UMAP(n_components = 2, n_neighbors=50, min_dist=0.4)
x_umap = u.fit_transform(data_tfidf)

ratings = list(df['Rating'])
reviews = list(df['Review'])
data = [go.Scatter(x=x_umap[:,0], y=x_umap[:,1], mode='markers',
                    marker = dict(color=Y, colorscale='Rainbow', opacity=0.5),
                                text=[f'Rating: {a}<br>Review: {b}' for a,b in list(zip(ratings,reviews))],
                                hoverinfo='text')]

layout = go.Layout(title = 'UMAP Dimensionality Reduction', width = 1400, height = 1400,
                    xaxis = dict(title='First Dimension'),
                    yaxis = dict(title='Second Dimension'))
fig = go.Figure(data=data, layout=layout)
fig.show()

The hyperparameters ngram_range and min_df in the TF-IDF vectorization process affect the TF-IDF matrix and the subsequent visualization as follows:

ngram_range: It refers to the range of n-grams (i.e., a contiguous sequence of n items from a given sample of text or speech) to consider while generating the TF-IDF matrix. By default, it is set to (1,1), which means only unigrams will be considered. However, by changing the value of ngram_range, we can include bigrams, trigrams, or more. Including bigrams or trigrams can help in capturing more context and better understanding of the text.

min_df: It refers to the minimum document frequency for a term to be considered while generating the TF-IDF matrix. It specifies the threshold for the frequency of the term in the corpus. By default, it is set to 1, which means all terms are considered. However, by changing the value of min_df, we can remove terms that occur rarely in the corpus. Setting a higher value of min_df will remove the rare terms from the matrix, and setting a lower value will keep more terms in the matrix.

Changing these hyperparameters affects the TF-IDF matrix and the visualization in the following ways:

Changing the ngram_range will change the size of the TF-IDF matrix. Including bigrams or trigrams will result in a larger matrix, and including unigrams will result in a smaller matrix. The number of features extracted from the corpus also increases with the increase in ngram_range. This can lead to a better understanding of the text as it includes more context, but it can also increase the noise in the data.

Changing the min_df will change the number of terms in the TF-IDF matrix. Increasing the min_df value will remove terms that occur rarely in the corpus, resulting in a smaller matrix. This can help in reducing noise and making the matrix more meaningful. However, it can also lead to information loss, as rare terms that might be relevant to the data are removed.

In the given code, the ngram_range is set to (1,4), which means unigrams, bigrams, trigrams, and 4-grams are considered. The min_df is set to 0.0005, which means a term must be present in at least 20 documents to be considered as a token. Not much has changed in visualization. We can still alter the values to get better visualizations.

----------------------------------------------------------------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------------------------------------------------------------

3. How does tuning UMAP hyperparameters ‘n_neighbors’ and ‘min_dist’ affect the
visualization? 


In [9]:
df['cleaned_review'] = df.Review.apply(cleaner)
df = df[df['cleaned_review'].map(len) > 0] # removing rows with cleaned reviews of length 0
print("Printing top 5 rows of dataframe showing original and cleaned reviews....")
print(df[['Review','cleaned_review']].head())
df['cleaned_review'] = [" ".join(row) for row in df['cleaned_review'].values] # joining tokens to create strings. TfidfVectorizer does not accept tokens as input
data = df['cleaned_review']
Y = df['Rating'] 
tfidf = TfidfVectorizer(min_df=0.00096 , ngram_range=(1,4)) 
tfidf.fit(data) # learn vocabulary of entire data
data_tfidf = tfidf.transform(data) # creating tfidf values
print("The created tokens: \n", tfidf.get_feature_names_out())
print("Shape of tfidf matrix: ", data_tfidf.shape)
print(type(data_tfidf))

Printing top 5 rows of dataframe showing original and cleaned reviews....
                                                                                                                                                                                                      Review  \
0     The Gourmet Kitchen serves the most delicious French cuisine in town! The ambience is perfect for a romantic dinner or a night out with friends. I highly recommend the escargot and the crème brûlée.   
1                              Mama's Italian Kitchen is my new favorite Italian restaurant! The pasta is always cooked perfectly, and the sauce is rich and flavorful. The garlic bread is also a must-try.   
2          Sushi Palace is the best sushi restaurant in the city. The sushi is always fresh and expertly prepared, and the miso soup is the perfect starter. The atmosphere is also very cozy and welcoming.   
3  The Green Table is the perfect spot for vegetarians and vegans. The food is always fresh an

In [30]:
u = umap.UMAP(n_components = 2, n_neighbors=65, min_dist=0.5)
x_umap = u.fit_transform(data_tfidf)

ratings = list(df['Rating'])
reviews = list(df['Review'])
data = [go.Scatter(x=x_umap[:,0], y=x_umap[:,1], mode='markers',
                    marker = dict(color=Y, colorscale='Rainbow', opacity=0.5),
                                text=[f'Rating: {a}<br>Review: {b}' for a,b in list(zip(ratings,reviews))],
                                hoverinfo='text')]
layout = go.Layout(title = 'UMAP Dimensionality Reduction', width = 1400, height = 1400,
                    xaxis = dict(title='First Dimension'),
                    yaxis = dict(title='Second Dimension'))
fig = go.Figure(data=data, layout=layout)
fig.show()

UMAP is an unsupervised dimensionality reduction technique that is used for visualizing high-dimensional data. The hyperparameters of UMAP, namely n_neighbors and min_dist, can affect the visualization output.

n_neighbors controls the size of the local neighborhood in the high-dimensional space, and min_dist controls the minimum distance between points in the low-dimensional space. Tuning these hyperparameters can affect the visual structure of the embedding.

In the provided code, UMAP is first trained with n_neighbors=65 and min_dist=0.5. The resulting visualization shows distinct clusters of points based on their ratings, and the clusters are well-separated. However, some points are overlapping, which can be due to the use of a lower value for min_dist.

When the value of n_neighbors is decreased to 30, the clusters become less distinct, and some points start to overlap, especially in the regions where different clusters are close to each other. On the other hand, when the value of n_neighbors is increased to 100, the clusters become more distinct, but some isolated points remain far from their respective clusters.

When the value of min_dist is increased to 0.8, the overlapping points are further apart, resulting in better separation of the clusters. However, the clusters become less dense, and some smaller clusters merge with the larger ones.

Overall, tuning the hyperparameters of UMAP can significantly affect the visualization output. A careful selection of hyperparameters can result in a more informative visualization. In the given code, n_neighbors=65 and min_dist=0.5 seem to provide a good balance between cluster separation and density.

4. Can you identify two clusters of reviews based on positive and negative sentiments? If yes,
can you identify any sub-clusters within these two clusters? If yes, what do the sub-clusters
tell us? 

Based on the visualization, it is apparent that there are discernible clusters of reviews with positive, neutral, and negative sentiments. Additionally, a smaller but still noticeable sub-cluster can be observed, which contains a mix of negative and neutral reviews. This could be due to the fact that the language used in these reviews is similar and certain words that are used to describe the restaurant may be common across these reviews, even though the ratings assigned to them vary. 